# PTT5 integration File

In [1]:
K_Passages = 5

path = '/mnt/Files/integration/ptt5/'

model_checkpoint = 'PTT5_Biomedical/checkpoint-8500'


## Loading trained PTT5

In [2]:
max_input_length = 512
max_target_length = 128



In [3]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [4]:
from transformers import AutoTokenizer, T5Tokenizer, T5TokenizerFast, TFT5Model, TFT5ForConditionalGeneration

#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

In [5]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [6]:
args = Seq2SeqTrainingArguments(
    "test-model",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    fp16=True,
    gradient_accumulation_steps = 4,
    predict_with_generate=True,
)
trainer = Seq2SeqTrainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer)

## Function that returns the answer to the question

In [7]:
import pandas as pd
from datasets import load_dataset, load_metric, DatasetDict, Dataset

def preprocess_input(examples):
    model_inputs = tokenizer(examples["question"], max_length=max_input_length, truncation=True)
    return model_inputs

def preprocess_function(examples):
    model_inputs = preprocess_input(examples)
    # Setup the tokenizer for targets
    labels = tokenizer(examples["answer"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def get_answer(question):
    df = pd.DataFrame([[question, "isso é um teste"]], columns = ["question", "answer"])

    dataset_test = Dataset.from_pandas(df=df)
    raw_datasets = DatasetDict({
        'test': dataset_test})
    
    tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)


    a = trainer.predict(tokenized_datasets['test'], max_length=max_target_length)
    test_outputs = []
    test_labels = []
    for i in range(len(a.predictions)):
        test_outputs.append(tokenizer.decode(a.predictions[i], skip_special_tokens=True))
        test_labels.append(raw_datasets['test'][i]['answer'])
    return(test_outputs[0])

## Main loop that receives a question and sends back the answer

In [ ]:
#aguarda pergunta e responde a resposta (inferência)

import csv
import time
import os
import ast


print(f"Listening path: {path}")

while 1:#loop infinito tem que parar manualmente
    time.sleep(0.5)#em segundos, delay para não consumir todo o processador da máquina.
    
    ready = os.path.exists(path + "in/ready.txt")
    question = os.path.exists(path + "in/question.csv")
    #print(f"Ready = {ready}\nQuestion = {question}")
        
    if ready == True and question == True:
        print("\nNew request arrive:")
                
        csv_out = open(path + "out/answer.csv", "w", encoding='utf8')
        ready_out = open(path + "out/ready.txt", "w", encoding='utf8')
        ready_out.write("Done")#qualquer conteúdo       

        with open(path + "in/question.csv", newline = '', encoding='utf8') as csv_in:
            csv_in_reader = csv.reader(csv_in, delimiter=';')
            for csv_i in csv_in_reader:
                print(csv_i[0])
                code = get_answer(csv_i[0])               
                print(code)
                csv_out.write(f"{csv_i[0]};{code}\n")
                break #só o primeiro               
        csv_out.close()   
        ready_out.close()
        os.remove(path + "in/ready.txt") #Deletado indica que terminou o processamento
        os.remove(path + "in/question.csv")#Deletado indica que terminou o processamento

Listening path: /mnt/Files/integration/ptt5/

New request arrive:
question: O que é transtorno obsessivo compulsivo [SEP] context: Definição médica de transtorno obsessivo-compulsivo. Transtorno obsessivo-compulsivo: um distúrbio psiquiátrico caracterizado por pensamentos obsessivos e ações compulsivas, como limpeza, verificação, contagem ou acumulação. Transtorno obsessivo-compulsivo (OCD), um dos transtornos de ansiedade, é uma condição potencialmente incapacitante que pode persistir ao longo da vida de uma pessoa. O transtorno obsessivo-compulsivo (OCD) se enquadra em uma categoria de condições chamadas distúrbios obsessivos-compulsivos e relacionados que incluem: 1 Transtorno Obsessivo-Compulsivo (TOC). 2 Transtorno Dismórfico Corporal. 3 desordem de acumulação. A coleta é um distúrbio que pode estar presente por conta própria ou como um sintoma de outro distúrbio. Aqueles que mais frequentemente associados à acumulação são transtorno de personalidade obsessivo-compulsivo (OCPD), t

  0%|          | 0/1 [00:00<?, ?ba/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Um distúrbio psiquiátrico caracterizado por pensamentos obsessivos e ações compulsivas, como limpeza, verificação, contagem ou acumulação.

New request arrive:
question: O que é transtorno obsessivo compulsivo [SEP] context: Definição médica de transtorno obsessivo-compulsivo. Transtorno obsessivo-compulsivo: um distúrbio psiquiátrico caracterizado por pensamentos obsessivos e ações compulsivas, como limpeza, verificação, contagem ou acumulação. Transtorno obsessivo-compulsivo (OCD), um dos transtornos de ansiedade, é uma condição potencialmente incapacitante que pode persistir ao longo da vida de uma pessoa. O transtorno obsessivo-compulsivo (OCD) se enquadra em uma categoria de condições chamadas distúrbios obsessivos-compulsivos e relacionados que incluem: 1 Transtorno Obsessivo-Compulsivo (TOC). 2 Transtorno Dismórfico Corporal. 3 desordem de acumulação. A coleta é um distúrbio que pode estar presente por conta própria ou como um sintoma de outro distúrbio. Aqueles que mais frequen

  0%|          | 0/1 [00:00<?, ?ba/s]

...

4 a 5 dias

New request arrive:
question: quanto tempo depois da gripe eu sou infeccioso [SEP] context: Resposta: Então, se você estiver perto de alguém que tenha a gripe, e no dia seguinte você se sente muito bem, você não está fora da floresta ainda. Você pode realmente desenvolver sintomas de até quatro a cinco dias após ser exposto a alguém que tenha a gripe. Próximo: Por quanto tempo eu sou contagiante depois de ser infectado pela gripe? Toque Confiança Votos 202. Vai doer tê-lo puxado quando é abscesso, mas se sentirá melhor muito logo depois. Eu tinha um puxado enquanto estava abscesso e era muito doloroso, mas eu sou muito grato que eu fiz isso. Eu não poderia ter ficado a dor do abscesso por muito mais tempo. Cura após uma extração de dentes pode levar até uma semana. Quanto tempo dura uma pessoa com a gripe contagiosa? Você pode passar a gripe para os outros de 1 dia antes de ter sintomas até 5-7 dias depois de ficar doente. Algumas pessoas podem infectar os outros ainda mais

  0%|          | 0/1 [00:00<?, ?ba/s]

4 a 5 dias

New request arrive:
question: O que causa dor nas costas [SEP] context: Sintomas de dor nas costas. Sintomas que geralmente ocorrem com a dor nas costas incluem rigidez traseira, espasmos musculares e ternura sobre os músculos das costas. Sintomas que indicam uma causa séria para a dor nas costas incluem dor nas costas intensa que atira a perna, dormência, fraqueza da perna, dormência da virilha e dificuldade em urinar. Em casos de dor nas costas crônicas, pode pensar: "As úlceras causam dor nas costas?" Uma resposta simples a essa pergunta ansiosa é "sim", uma úlcera péptica é uma provável causa de dor nas costas. Dor nas costas da moeda aguda. A dor nas costas da moeda aguda é geralmente o resultado de uma lesão muscular súbita ou superexerção. Em praticamente todos os casos, a dor é precedida por algum evento que realmente causa trauma ou é percebido para causar ferimentos. A dor nas costas pode vir de repente e durar menos de seis semanas (aguda), que pode ser causada p

  0%|          | 0/1 [00:00<?, ?ba/s]

A dor nas costas é causada por uma queda ou levantamento pesado.
